<a href="https://colab.research.google.com/github/Sivakumar793/Gen-AI-with-LSTM---Text-Generation/blob/main/GenAI_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import json
import requests
import random
from tqdm import tqdm
from collections import Counter

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
import nltk
from nltk.tokenize import word_tokenize

In [4]:
# 0) Settings (tweak here)
# ---------------------------
TXT_PATH = "shakespeare_complete_works.txt"
CHECKPOINT_PATH = "best_lstm_shakespeare.h5"
WORD_TO_ID_PATH = "word_to_id.json"
ID_TO_WORD_PATH = "id_to_word.json"

In [5]:
# Memory-friendly defaults (adjust if you have lots of RAM/GPU)
VOCAB_SIZE = 5000       # most frequent tokens to keep
SEQ_LENGTH = 30         # input words per sequence
STEP = 3                # sliding window step (larger -> fewer sequences)
EMBEDDING_DIM = 128
LSTM_UNITS = 128
BATCH_SIZE = 64
EPOCHS = 5
PATIENCE = 3            # early stopping patience

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [7]:
# 1) Download dataset (if missing)
# ---------------------------
if not os.path.exists(TXT_PATH):
    print("📥 Downloading Shakespeare (Project Gutenberg)...")
    url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"
    r = requests.get(url, timeout=30)
    r.encoding = "utf-8"
    with open(TXT_PATH, "w", encoding="utf-8") as f:
        f.write(r.text)
    print("Saved to", TXT_PATH)
else:
    print("Found local", TXT_PATH)

Found local shakespeare_complete_works.txt


In [8]:
# 2) Load & preprocess
# ---------------------------
with open(TXT_PATH, "r", encoding="utf-8") as f:
    text = f.read()

# Lowercase and strip Gutenberg header/footer if present
text = text.lower()
start_marker = "*** start of this project gutenberg ebook"
end_marker = "*** end of this project gutenberg ebook"
si = text.find(start_marker)
if si != -1:
    text = text[si + len(start_marker):]
ei = text.find(end_marker)
if ei != -1:
    text = text[:ei]

# Remove control chars and many punctuation we don't want
text = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', text)
text = re.sub(r'[_\*\[\]\(\)\{\}<>\/\\"=+#@\$%&\^~\|`]', ' ', text)

In [9]:
# 3) Tokenize
# ---------------------------
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

tokens = word_tokenize(text)
# keep tokens with letters or digits or apostrophes (filter noisy punctuation)
tokens = [t for t in tokens if re.search(r"[a-z0-9']", t)]

print("Total tokens:", len(tokens))
print("Example tokens:", tokens[:30])

Total tokens: 988070
Example tokens: ['\ufeffthe', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the']


In [10]:
# 4) Vocabulary (limit vocab)
# ---------------------------
counter = Counter(tokens)
most_common = counter.most_common(VOCAB_SIZE)
vocab = [w for w, _ in most_common]

# reserve:
#   0 -> padding (if needed)
#   1..len(vocab) -> vocab tokens
#   unknown_token_id = len(vocab) + 1
word_to_id = {w: i + 1 for i, w in enumerate(vocab)}
unknown_token_id = len(vocab) + 1
id_to_word = {str(i + 1): w for i, w in enumerate(vocab)}  # keys as strings for JSON

print("Vocabulary kept:", len(vocab))
print("Unknown token id:", unknown_token_id)

Vocabulary kept: 5000
Unknown token id: 5001


In [11]:
# 5) Encode tokens to ids
# ---------------------------
token_ids = [word_to_id.get(t, unknown_token_id) for t in tokens]

In [12]:
# 6) Create input-output sequences (memory-friendly sampling)
# ---------------------------
sequences = []
next_words = []
for i in range(0, len(token_ids) - SEQ_LENGTH, STEP):
    seq = token_ids[i:i + SEQ_LENGTH]
    nxt = token_ids[i + SEQ_LENGTH]
    sequences.append(seq)
    next_words.append(nxt)

print("Total sequences:", len(sequences))

# Convert to numpy arrays (integers, not one-hot)
X = np.array(sequences, dtype=np.int32)
y = np.array(next_words, dtype=np.int32)

Total sequences: 329347


In [13]:
# 7) Train / validation split
# ---------------------------
split_at = int(0.8 * len(X))
X_train, X_val = X[:split_at], X[split_at:]
y_train, y_val = y[:split_at], y[split_at:]

print("Train sequences:", X_train.shape, "Validation sequences:", X_val.shape)

Train sequences: (263477, 30) Validation sequences: (65870, 30)


In [14]:
# 8) Build model (sparse labels)
# ---------------------------
num_classes = unknown_token_id + 1  # include unknown and optionally 0 pad

model = Sequential([
    Embedding(input_dim=num_classes, output_dim=EMBEDDING_DIM, input_length=SEQ_LENGTH),
    LSTM(LSTM_UNITS, return_sequences=False),
    Dropout(0.2),
    Dense(num_classes, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",  # use sparse labels (integers)
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
# 9) Callbacks
# ---------------------------
checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor="val_loss", save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True, verbose=1)


In [16]:
# 10) Option A: Use tf.data pipeline (recommended for larger datasets)
# ---------------------------
# Convert to tf.data.Dataset for efficient batching/prefetching
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=10000, seed=RANDOM_SEED).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [17]:
# 11) Train
# ---------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint, earlystop],
    verbose=1
)

print("Training finished. Best weights (if improved) saved to:", CHECKPOINT_PATH)

Epoch 1/5
4116/4117 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0706 - loss: 6.2265
Epoch 1: val_loss improved from inf to 6.02256, saving model to best_lstm_shakespeare.h5


4117/4117 ━━━━━━━━━━━━━━━━━━━━ 263s 63ms/step - accuracy: 0.0706 - loss: 6.2264 - val_accuracy: 0.0983 - val_loss: 6.0226
Epoch 2/5
4116/4117 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1038 - loss: 5.5967
Epoch 2: val_loss improved from 6.02256 to 5.91325, saving model to best_lstm_shakespeare.h5


4117/4117 ━━━━━━━━━━━━━━━━━━━━ 263s 64ms/step - accuracy: 0.1038 - loss: 5.5967 - val_accuracy: 0.1115 - val_loss: 5.9132
Epoch 3/5
4117/4117 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1203 - loss: 5.3596
Epoch 3: val_loss improved from 5.91325 to 5.90397, saving model to best_lstm_shakespeare.h5


4117/4117 ━━━━━━━━━━━━━━━━━━━━ 262s 64ms/step - accuracy: 0.1203 - loss: 5.3596 - val_accuracy: 0.1182 - val_loss: 5.9040
Epoch 4/5
4117/4117 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1294 - loss: 5.1964
Epoch 4: val_loss improved from 5.90397 to 5.90269, saving model to best_lstm_shakespeare.h5


4117/4117 ━━━━━━━━━━━━━━━━━━━━ 262s 64ms/step - accuracy: 0.1294 - loss: 5.1964 - val_accuracy: 0.1205 - val_loss: 5.9027
Epoch 5/5
4116/4117 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1366 - loss: 5.0641
Epoch 5: val_loss did not improve from 5.90269
4117/4117 ━━━━━━━━━━━━━━━━━━━━ 322s 64ms/step - accuracy: 0.1366 - loss: 5.0640 - val_accuracy: 0.1233 - val_loss: 5.9119
Restoring model weights from the end of the best epoch: 4.
Training finished. Best weights (if improved) saved to: best_lstm_shakespeare.h5


In [18]:
# 12) Text generation utilities
# ---------------------------

def id_to_word_safe(idx):
    """
    idx: integer index
    returns readable token string
    """
    if idx == 0:
        return ""
    if str(idx) in id_to_word:
        return id_to_word[str(idx)]
    if idx == unknown_token_id:
        return "<UNK>"
    return "<PAD>"

In [19]:
def sample_with_temperature(preds, temperature=1.0):
    """
    preds: 1D numpy array of probabilities (model output)
    temperature: float - higher -> more random
    returns index (int) sampled from distribution
    """
    preds = np.asarray(preds).astype('float64')
    # prevent log(0)
    preds = np.clip(preds, 1e-20, 1.0)
    log_preds = np.log(preds) / (temperature if temperature > 0 else 1e-9)
    exp_preds = np.exp(log_preds)
    probs = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(probs), p=probs)

In [20]:
def generate_text(model, seed_text, length=100, temperature=1.0):
    """
    Generate `length` words given seed_text.
    """
    model_input = []
    seed_tokens = word_tokenize(seed_text.lower())
    seed_ids = [word_to_id.get(t, unknown_token_id) for t in seed_tokens]

    # left-pad with zeros if seed shorter than SEQ_LENGTH
    if len(seed_ids) < SEQ_LENGTH:
        seed_ids = [0] * (SEQ_LENGTH - len(seed_ids)) + seed_ids
    else:  # truncate to last SEQ_LENGTH
        seed_ids = seed_ids[-SEQ_LENGTH:]

    generated = seed_tokens[:]  # keep human-readable tokens

    for _ in range(length):
        x = np.array(seed_ids, dtype=np.int32).reshape(1, -1)
        preds = model.predict(x, verbose=0)[0]  # shape -> (num_classes,)

        next_id = sample_with_temperature(preds, temperature=temperature)
        next_word = id_to_word_safe(next_id)
        generated.append(next_word)

        # slide window
        seed_ids = seed_ids[1:] + [next_id]

    # join carefully: remove empty tokens
    return " ".join([w for w in generated if w])


In [21]:
# If checkpoint exists, load best weights
if os.path.exists(CHECKPOINT_PATH):
    try:
        model.load_weights(CHECKPOINT_PATH)
        print("Loaded weights from", CHECKPOINT_PATH)
    except Exception as e:
        print("Could not load checkpoint:", e)

Loaded weights from best_lstm_shakespeare.h5


In [22]:
# 13) Example generations
# ---------------------------
seeds = [
    "the king said",
    "to be or not to be",
    "in the play",
    "o my lord"
]

for seed in seeds:
    print("\nSEED:", seed)
    sample = generate_text(model, seed, length=80, temperature=0.8)
    print(sample)
    print("-" * 80)


SEED: the king said
the king said not <UNK> capulet o my lord and <UNK> no greater and <UNK> against the world i shall make it his creature i ll make me juliet i ll tell a piece of my <UNK> <UNK> tranio my lord sir my lord is and <UNK> for your daughter s tears and <UNK> them <UNK> <UNK> too <UNK> and do i ll go to prison petruchio a fashion i did in <UNK> sebastian i that labour will have show a paper and
--------------------------------------------------------------------------------

SEED: to be or not to be
to be or not to be <UNK> d to the moon to send on hot hours and lusty <UNK> though they may see me and to the <UNK> of your <UNK> and make our company and all the <UNK> degree on <UNK> <UNK> and <UNK> had <UNK> <UNK> in the holy shore which <UNK> the <UNK> so now and at <UNK> of them therefore will <UNK> <UNK> of thee a spirit and fury d and all <UNK> not to can <UNK> my fortunes again i know
--------------------------------------------------------------------------------

SEED

In [23]:
# 14) Save vocab mappings (JSON-friendly)
# ---------------------------
# word_to_id: keys are words (strings) -> values are ints
with open(WORD_TO_ID_PATH, "w", encoding="utf-8") as f:
    json.dump(word_to_id, f, ensure_ascii=False, indent=2)

# id_to_word keys are strings already (stringified ids) -> values are words
with open(ID_TO_WORD_PATH, "w", encoding="utf-8") as f:
    json.dump(id_to_word, f, ensure_ascii=False, indent=2)

print("Saved vocab mappings to", WORD_TO_ID_PATH, "and", ID_TO_WORD_PATH)
print("Done.")

Saved vocab mappings to word_to_id.json and id_to_word.json
Done.
